# Connect to Crunchy Bridge Cloud using pyservicebinding

## Install prerequisite libraries for Service Binding and PostgreSQL (Crunchy Bridge) API access.

In [15]:
!pip install pyservicebinding
!pip install psycopg2-binary

## Check if the Service Binding has been setup.

In [2]:
import os
import pyservicebinding
from pyservicebinding import binding

if 'SERVICE_BINDING_ROOT' in os.environ:
    print('DB binding ok')
else:
    print('DB binding missing')

DB binding ok


## Using pyservicebinding library get credentials for Crunchy Bridge Cloud.

In [3]:
sb = binding.ServiceBinding()
bindings_list = sb.bindings('postgresql', 'Red Hat DBaaS / Crunchy Bridge')
if (len(bindings_list)) > 0:
    print('Crunchy Bridge instance - found')
else:
    print('Crunchy Bridge instance - not found')

Crunchy Bridge instance - found


## Use the credentials from the service binding to connect to Crunchy Bridge Cloud.

In [19]:
import psycopg2
import psycopg2.errorcodes
import time
import logging
import random
db_connection = psycopg2.connect(database=bindings_list[0].get('database'), \
    user=bindings_list[0].get('username'), \
    password=bindings_list[0].get('password'), \
    host=bindings_list[0].get('host'), \
    port=bindings_list[0].get('port'))
print(db_connection)

<connection object at 0x7f33ff3cab80; dsn: 'user=postgres password=xxx dbname=postgres host=p.ijkdpdsaqzhgzeleiv4ousk5hi.db.postgresbridge.com port=5432', closed: 0>


## On the connected Postgres database, run a SQL query on the *bluebook_small* table <br>Ingest query results into a Pandas dataframe and perform feature selection using Scikit-learn ML library.

In [5]:
import pandas as pd
cur = db_connection.cursor()
cur.execute("SELECT * FROM bluebook_small")
logging.debug("select_all(): status message: {}".format(cur.statusmessage))
df = pd.DataFrame(cur.fetchall(), columns = ['Price','Mileage','Make','Model','Trim','Type','Cylinder','Liter','Doors','Cruise','Sound','Leather'])
df.head()

,Price,Mileage,Make,Model,Trim,Type,Cylinder,Liter,Doors,Cruise,Sound,Leather
0,17314.10313,8221,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,1
1,17542.03608,9135,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,0
2,16218.84786,13196,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,0
3,16336.91314,16342,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,0,0
4,16339.17032,19832,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,0,1


In [6]:
import seaborn as sns
import csv
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import zipfile
from IPython.display import display, Latex, Markdown
#import plotly.express as px
#import plotly.graph_objs as go
plt.rcParams['font.sans-serif'] = "Comic Sans MS"
import warnings; warnings.simplefilter('ignore')
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline

## The categorical features need to be mapped to numerical labels using sklearn's LabelEncoder.

In [7]:
from sklearn.preprocessing import LabelEncoder 
  
le = LabelEncoder()
df['Make']= le.fit_transform(df['Make']) 
df['Model']= le.fit_transform(df['Model']) 
df['Trim']= le.fit_transform(df['Trim']) 
df['Type']= le.fit_transform(df['Type'])
df

,Price,Mileage,Make,Model,Trim,Type,Cylinder,Liter,Doors,Cruise,Sound,Leather
0,17314.10313,8221,0,10,44,3,6,3.1,4,1,1,1
1,17542.03608,9135,0,10,44,3,6,3.1,4,1,1,0
2,16218.84786,13196,0,10,44,3,6,3.1,4,1,1,0
3,16336.91314,16342,0,10,44,3,6,3.1,4,1,0,0
4,16339.17032,19832,0,10,44,3,6,3.1,4,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
799,16507.07027,16229,5,21,21,3,6,3,4,1,0,0
800,16175.9576,19095,5,21,21,3,6,3,4,1,1,0
801,15731.1329,20484,5,21,21,3,6,3,4,1,1,0
802,15118.89323,25979,5,21,21,3,6,3,4,1,1,0


## Splitting data into feature columns (X data) and our target column (y data).

In [8]:
X = df.drop('Price', 1)
X = X.apply(pd.to_numeric)
y = df['Price']
y = y.apply(pd.to_numeric)

## Sklearn’s StandardScaler can be used to normalize the features as part of the preparation for modeling.

In [9]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
X[['Mileage', 'Make', 'Model', 'Trim', 'Type', 'Cylinder', 'Liter', 'Doors', 'Cruise', 'Sound', 'Leather']] = scale.fit_transform(X[['Mileage', 'Make', 'Model', 'Trim', 'Type', 'Cylinder', 'Liter', 'Doors', 'Cruise', 'Sound', 'Leather']].values)

In [10]:
X

,Mileage,Make,Model,Trim,Type,Cylinder,Liter,Doors,Cruise,Sound,Leather
0,-1.417485,-1.802394,-0.556062,1.203142,0.498507,0.52741,0.056736,0.556279,0.57352,0.687406,0.617611
1,-1.305902,-1.802394,-0.556062,1.203142,0.498507,0.52741,0.056736,0.556279,0.57352,0.687406,-1.619142
2,-0.810128,-1.802394,-0.556062,1.203142,0.498507,0.52741,0.056736,0.556279,0.57352,0.687406,-1.619142
3,-0.426058,-1.802394,-0.556062,1.203142,0.498507,0.52741,0.056736,0.556279,0.57352,-1.454744,-1.619142
4,0.000008,-1.802394,-0.556062,1.203142,0.498507,0.52741,0.056736,0.556279,0.57352,-1.454744,0.617611
...,...,...,...,...,...,...,...,...,...,...,...
799,-0.439853,1.959613,0.708158,-0.396779,0.498507,0.52741,-0.033772,0.556279,0.57352,-1.454744,-1.619142
800,-0.089966,1.959613,0.708158,-0.396779,0.498507,0.52741,-0.033772,0.556279,0.57352,0.687406,-1.619142
801,0.079605,1.959613,0.708158,-0.396779,0.498507,0.52741,-0.033772,0.556279,0.57352,0.687406,-1.619142
802,0.750446,1.959613,0.708158,-0.396779,0.498507,0.52741,-0.033772,0.556279,0.57352,0.687406,-1.619142


## Recursive Feature Elimination (RFE) is a powerful feature selection tool to select a subset of k features given all the features available. In this example, we are using sklearn’s feature selection library and choosing a Decision Tree Regressor estimator for RFE. The reason why we are using regressors over classifiers is that we do not have a multi-class problem here; since we are predicting the price of cars based on its characteristics, we are minimizing the loss in the arbitrary loss function chosen to find the most suitable and optimizable model for this data science question. As standard practice suggests, we have randomly split the dataset into “unseen” test data and training data using train_test_split().

In [11]:
# evaluate RFE for regression
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline

# create pipeline
rfe = RFE(estimator=DecisionTreeRegressor(), n_features_to_select=5)
model = DecisionTreeRegressor()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
# evaluate model
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: -891.644 (311.006)


## We chose MAE (mean absolute error) as there is no difference between overestimating and underestimating the price of a car; we are simply seeing how different the delta is between the predicted and actual price of the car. The MAE of RFE comes to ~800.

In [12]:
feature_name = X.columns.tolist()
num_feats = len(feature_name)

In [13]:
# create pipeline
from sklearn.model_selection import train_test_split
rfe = RFE(estimator=DecisionTreeRegressor(), n_features_to_select=5)
model = DecisionTreeRegressor()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
# fit the model on all available data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
pipeline.fit(X_train, y_train)
yhat = pipeline.predict(X_test)
#print(yhat)
# summarize all features
for i in range(X_train.shape[1]):
    print('Column: %d, %s, Selected %s, Rank: %.3f' % (i, feature_name[i], rfe.support_[i], rfe.ranking_[i]))
print("PREDICTED VALUE\n", pd.DataFrame(data=yhat[:20]), "\nTEST VALUE\n", y_test[:20])

Column: 0, Mileage, Selected True, Rank: 1.000
Column: 1, Make, Selected True, Rank: 1.000
Column: 2, Model, Selected True, Rank: 1.000
Column: 3, Trim, Selected True, Rank: 1.000
Column: 4, Type, Selected False, Rank: 3.000
Column: 5, Cylinder, Selected False, Rank: 6.000
Column: 6, Liter, Selected True, Rank: 1.000
Column: 7, Doors, Selected False, Rank: 2.000
Column: 8, Cruise, Selected False, Rank: 7.000
Column: 9, Sound, Selected False, Rank: 4.000
Column: 10, Leather, Selected False, Rank: 5.000
PREDICTED VALUE
               0
0   15048.04218
1   16516.95555
2   15589.78048
3   44205.87588
4   12425.38917
5   27060.13756
6   15086.89959
7   29612.15405
8   30661.26015
9   38208.50102
10  13825.15471
11  20537.14172
12  19344.16554
13  33183.33260
14  27280.98206
15  16516.95555
16  25148.37919
17  23102.02237
18  20537.14172
19  14909.05099 
TEST VALUE
 8      14862.09387
575    15979.01484
616    15802.65286
142    45061.95194
236    12630.77503
664    25618.28188
608    15174.

In [ ]:
import plotly.express as px
import plotly.graph_objs as go
fig = go.Figure()
count = 20
fig.add_trace(go.Scatter(x = X_test[:count], y = y_test[:count], mode = 'markers', name = 'actual', marker=dict(size=10,color=1)))
fig.add_trace(go.Scatter(x = X_test[:count], y = yhat[:count], mode = 'markers', name = 'predictions', marker=dict(size=10,color=3)))
fig.update_layout( yaxis_title = 'Price')